In [1]:
# Atanas and Talha
import numpy as np 
from math import exp, log, sqrt
from scipy.optimize import minimize
from tabulate import tabulate
import networkx as nx
import matplotlib.pyplot as plt

r0 = 0.05034
r1 = 0.05030
dt = 0.25 # months, thus a period is 3 months
vol = 0.0025
drift = [0.0, 0.003, 0.005]
t = 2
a = [100.0*exp(-r0*dt*2.0), 100.0*exp(-r1*dt*3.0)]

rates = np.zeros((t,t))
for i in range (0,t):
    if i > 0:
        rates[t-1, i] = rates[t-1, i-1] + drift[i]*dt - vol*sqrt(dt)
    else:
        rates[t-1, i] = r0
    rates[t-2, i] = rates[t-1, i] + 2.0*vol*sqrt(dt)
    rates[t-3, i] = rates[t-2, i] + 2.0*vol*sqrt(dt)

print(tabulate(rates))

def tree(drift, r0, vol, dt):
    t = len(drift)
    rates = np.zeros((t,t))
    for j in range (0,t):
        for i in range (j, t):
            if i == 0 and j == 0:
                rates[t-1, i] = r0
            elif i > 0 and j == 0:
                rates[t-1, i] = rates[t-1, i-1] + drift[i]*dt - vol*sqrt(dt)
            else:
                rates[t-1-j, i] = rates[t-1, i] + 2.0*j*vol*sqrt(dt)

    return rates

print(tabulate(tree(drift, r0, vol, dt)))

t = 2
r6m = tree(drift[:t], r0, vol, dt)
zc6m = np.zeros((t,t))
for i in range (0,t):
    for j in range(i, t):
        if i == 0:
            zc6m[j, t-1-i] = 100.0*exp(-r6m[j, t-1-i]*dt)
        else:
            zc6m[j, t-1-i] = 0.5*(zc6m[j, t-i]+zc6m[j-1,t-i])*exp(-r6m[j, t-1-i]*dt)

#print(tabulate(zc6m))
#print(tabulate(r6m))

def zc(r6m, dt):
    t = r6m.shape[0]
    zc6m = np.zeros((t,t))
    for i in range (0,t):
        for j in range(i, t):
            if i == 0:
                zc6m[j, t-1-i] = 100.0*exp(-r6m[j, t-1-i]*dt)
            else:
                zc6m[j, t-1-i] = 0.5*(zc6m[j, t-i]+zc6m[j-1,t-i])*exp(-r6m[j, t-1-i]*dt)    

    return zc6m

print((tabulate(zc(r6m,dt))))
print(a)

def zc_err(drift, a, param, m_old):
    r0, vol, dt = param[0], param[1], param[2]
    t = len(m_old) + 1
    rr = tree(m_old.__add__([drift]), r0, vol, dt)
    zc6m = zc(rr, dt)
    return (a[t-2] - zc6m[t-1,0])**2

param = [r0, vol, dt]
m_old = [0.0, 3.126756413565513e-06]
print('Gap 1:', (zc(r6m, dt)[1,0]-a[1])**2)
print('Gap 2:', zc_err(0.0046893708761831425, a, param, m_old))
print(a)
print(drift)


-------  -------
0.05284  0.05734
0.05534  0.05984
-------  -------
-------  -------  -------
0        0        0.05484
0        0.05234  0.05234
0.05034  0.04984  0.04984
-------  -------  -------
-------  -------
 0       98.7
97.4961  98.7617
-------  -------
[97.51441234357175, 96.29777233558649]
Gap 1: 1.4360727538772649
Gap 2: 0.0009685495351384266
[97.51441234357175, 96.29777233558649]
[0.0, 0.003, 0.005]


In [2]:
params = [r0, vol, dt]
m_old = [0.0]
t = 2
for i in range(t):
    temp = minimize(zc_err,m_old[-1],args=(a,params,m_old),method="L-BFGS-B")
    m_old = m_old.__add__([temp.x[0]])
    temp_tree = tree(m_old,r0,vol,dt)
    print(m_old)
    print(tabulate(temp_tree))
    print(tabulate(zc(temp_tree,dt)))

[0.0, 7.779306187175263e-07]
-------  ---------
0        0.0515902
0.05034  0.0490902
-------  ---------
-------  -------
 0       98.7185
97.5144  98.7802
-------  -------
[0.0, 7.779306187175263e-07, -0.00047765523876201857]
-------  ---------  ---------
0        0          0.0527208
0        0.0515902  0.0502208
0.05034  0.0490902  0.0477208
-------  ---------  ---------
-------  -------  -------
 0        0       98.6906
 0       97.4564  98.7523
96.2978  97.5783  98.8141
-------  -------  -------


/var/folders/ww/8s87d_9d7qz3xts57gbrj_ph0000gn/T/ipykernel_12357/2934102777.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rates[t-1, i] = rates[t-1, i-1] + drift[i]*dt - vol*sqrt(dt)


In [11]:
rateslist = [0.0534, 0.0520, 0.0501, 0.0480, 0.0460, 0.0442, 0.0426, 0.0414, 0.0406, 0.04, 0.0394, 0.0391,
              0.0388, 0.0387, 0.0386, 0.0386, 0.0386, 0.0387, 0.0389, 0.0392, 0.0394, 0.0397, 0.04, 0.0402]
ts = np.arange(1,25)*dt
bonds = 100.0/np.exp(rateslist*ts)
t = 24
params = [r0, vol, dt]
m_old = [0.0]
for i in range(t):
    params = [r0, vol, dt]
    temp = minimize(zc_err,m_old[-1],args=(bonds,params,m_old),method="L-BFGS-B")
    m_old = m_old.__add__([temp.x[0]])
    temp_tree = tree(m_old,r0,vol,dt)
    G = nx.DiGraph()
    for node in temp_tree:
        node_tuple = tuple(node.tolist())  # Convert numpy array to tuple
        G.add_node(node_tuple)
        if not np.array_equal(node, np.array([0.0])):  # Skip if the node is the root
            parent = tuple((node - 1) // 2)
            G.add_edge(parent, node_tuple)
    


    zc_tree = zc(temp_tree, dt)
    G_zc = nx.DiGraph()
    for node in zc_tree:
        node_tuple = tuple(node.tolist())  # Convert numpy array to tuple
        G_zc.add_node(node_tuple)
        if not np.array_equal(node, np.array([0.0])):  # Skip if the node is the root
            parent = tuple((node - 1) // 2)
            G_zc.add_edge(parent, node_tuple)
    

    print(tabulate(temp_tree))
    print(tabulate(zc_tree))
print(bonds)
print(m_old)


-------  ---------
0        0.0043102
0.05034  0.0018102
-------  ---------
-------  -------
 0       99.8923
98.6739  99.9548
-------  -------
-------  ---------  ---------
0        0          0.0531008
0        0.0043102  0.0506008
0.05034  0.0018102  0.0481008
-------  ---------  ---------
-------  -------  -------
 0        0       98.6813
 0       98.6058  98.7429
97.4335  98.7291  98.8047
-------  -------  -------
-------  ---------  ---------  ---------
0        0          0          0.0500518
0        0          0.0531008  0.0475518
0        0.0043102  0.0506008  0.0450518
0.05034  0.0018102  0.0481008  0.0425518
-------  ---------  ---------  ---------
-------  -------  -------  -------
 0        0        0       98.7565
 0        0       97.4846  98.8182
 0       97.4405  97.6065  98.88
96.3122  97.6234  97.7286  98.9418
-------  -------  -------  -------
-------  ---------  ---------  ---------  ---------
0        0          0          0          0.0467031
0        0        

/var/folders/ww/8s87d_9d7qz3xts57gbrj_ph0000gn/T/ipykernel_12357/2934102777.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  rates[t-1, i] = rates[t-1, i-1] + drift[i]*dt - vol*sqrt(dt)
